In [ ]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.model.PermutationFeatureImportance import permutation_featue_importance

In [ ]:
h2o.init()

In [ ]:
from h2o.utils.shared_utils import _locate # private function. used to find files within h2o git project directory.

df = h2o.import_file(path=_locate("smalldata/logreg/prostate.csv"))

In [ ]:
df.describe()

In [ ]:
# Remove ID from training frame
train = df.drop("ID")

In [ ]:
# For VOL & GLEASON, a zero really means "missing"
vol = train['VOL']
vol[vol == 0] = None
gle = train['GLEASON']
gle[gle == 0] = None

In [ ]:
# Convert CAPSULE to a logical factor
train['CAPSULE'] = train['CAPSULE'].asfactor()

In [ ]:
# See that the data is ready
train.describe()

In [ ]:
# Run GBM
my_gbm = H2OGradientBoostingEstimator(distribution = "bernoulli", ntrees=50, learn_rate=0.1)

my_gbm.train(x=list(range(1,train.ncol)), y="CAPSULE", training_frame=train, validation_frame=train)

In [1]:
pfi_fr =permutation_featue_importance(train, my_gbm)

pfi_fr

NameError: name 'permutation_featue_importance' is not defined